Notebook for gridding AEM using gdal

In [1]:
%matplotlib inline

In [18]:
import netCDF4
import numpy as np
from hydrogeol_utils import spatial_functions
from geophys_utils._netcdf_point_utils import NetCDFPointUtils
import time
import rasterio
import affine

In [3]:
infile = r"C:\Users\PCUser\Desktop\AEM\LCI\HowardE_WB_MGA52.nc"

lci_dat = netCDF4.Dataset(infile)
cond_point_utils = NetCDFPointUtils(lci_dat)
utm_wkt, aem_coords = cond_point_utils.utm_coords(cond_point_utils.xycoords)

minx, maxx  = aem_coords[:,0].min(), aem_coords[:,0].max()
miny, maxy  = aem_coords[:,1].min(), aem_coords[:,1].max()

In [49]:
# Define gdal algorithm as string - see https://gdal.org/programs/gdal_grid.html
algorithm = 'invdist:power=2:radius1=250:radius2=250:max_points=15:'
algorithm += 'min_points=2:nodata=-9999.'

grid_kwargs = {'conductivity': {'log_grid': True,
                                'gdal_algorithm': algorithm}}

start_time = time.time()

# Currently this is too resource intense, need to make it more effecient
aem_grid = spatial_functions.grid_points_gdal(cond_point_utils,
                 grid_resolution = 20.,
                 variables = 'conductivity',
                 grid_wkt = utm_wkt,
                 reprojected_grid_bounds = (minx + 200., miny -200.,
                                            maxx + 200., maxy - 200.),
                 point_step=1, # Only use every 4th point
                 grid_kwargs = grid_kwargs)

print("Gridding time: ", time.time() - start_time, ' seconds')

Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finished gridding  temp_grid.tif
Gridding  temp_grid.tif
Finishe

In [50]:
newarr = aem_grid['conductivity']

In [51]:
# Convert gdal geotransform to affine


(c, a, b, f, d, e) = aem_grid['geotransform']

newaff = affine.Affine(a, b, c,
                      d, e, f)

In [52]:
outfile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\grids\HE_layer_grid.tif"

new_dataset = rasterio.open(outfile, 'w', driver='GTiff',
                                height=newarr.shape[1], width=newarr.shape[2],
                                count=newarr.shape[0], dtype=newarr.dtype,
                                crs=utm_wkt, transform=newaff)

for i, item in enumerate(newarr):
    new_dataset.write(item, i+1)
new_dataset.close()

In [44]:
new_dataset.close()

In [30]:
item.shape

(387, 469)

In [35]:
aem_grid['geotransform']

(718150.0, 50.0, 0.0, 8611900.0, 0.0, 50.0)